In [6]:
import sys
import os
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

import pandas as pd
from isolated_ad_model.processing import PREPROCESS
import logging 
from AnomalyModelFactory import AnomalyModelFactory
from AnomalyRobustnessEvaluator import AnomalyRobustnessEvaluator

logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s [%(levelname)s] %(message)s', 
    handlers=[logging.StreamHandler()]
)

logger = logging.getLogger(__name__)

In [7]:
#config
use_cached_model = True
model_name = "isolation_forest"
model_path = "model_" + model_name + ".pkl"


#obtain the dataset
dataset = pd.read_csv('../isolated_ad_model/ue.csv')
true_anomalies = dataset['Viavi.UE.anomalies']
ps = PREPROCESS(dataset)  # TODO: Is it possible to get rid of src/scale dependency? 
ps.process()
training_data = ps.data

#obtain the model
model_factory = AnomalyModelFactory() # TODO: Add configuration from file

if use_cached_model:
    model = model_factory.load_model(model_path) 
else:
    model = model_factory.build_model(training_data, true_anomalies, model_name)
    model_factory.save_model(model_path)

#testing enviroment
evaluator = AnomalyRobustnessEvaluator(model, training_data, true_anomalies)

In [8]:
evaluator.test_a2pm()
evaluator.test_hsja()

2025-04-08 12:41:55,452 [INFO] 
--- Before A2PM Attack ---
2025-04-08 12:41:55,453 [INFO] Inliers: 5700, Outliers: 4300
2025-04-08 12:41:55,470 [INFO] Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.74      0.84      7432
           1       0.56      0.93      0.70      2568

    accuracy                           0.79     10000
   macro avg       0.76      0.84      0.77     10000
weighted avg       0.86      0.79      0.80     10000

2025-04-08 12:41:55,481 [INFO] Macro F1: 0.7687
2025-04-08 12:41:55,483 [INFO] Confusion Matrix:
[[5523 1909]
 [ 177 2391]]
2025-04-08 12:41:55,488 [INFO] 
--- After A2PM Attack ---
2025-04-08 12:41:55,489 [INFO] Inliers: 36, Outliers: 9964
2025-04-08 12:41:55,509 [INFO] Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      7432
           1       0.25      0.99      0.40      2568

    accuracy                           0